In [2]:
import requests 
import json 
import pandas as pd
import time
import urllib
import urllib.request 
from urllib.error import HTTPError
from bs4 import *
import os
#!pip install unidecode
import unidecode

from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
import cv2
from PIL import Image, UnidentifiedImageError
from pdb import set_trace
import copy
from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io, transform
from tqdm import trange, tqdm
import csv 
import glob
import dlib
import numpy as np
import random
from requests.adapters import HTTPAdapter
import torch
import hashlib
import shutil
import tempfile
from urllib.request import urlopen, Request

!pip install facenet_pytorch
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face

use_cuda=torch.cuda.is_available()
device=torch.device("cuda" if use_cuda else "cpu")
print("Using %s device"%(device))

################3
#Global variables
###################
#Change the following path to the folder containing the images
data_dir=r"C:\Users\mihnea.andrei\Work\facial_recognition\images"
save_dir=r"C:\Users\mihnea.andrei\Work\facial_recognition\images"
temp_dir=r"C:\Users\mihnea.andrei\Work\facial_recognition\temp"
temp_parent=r"C:\Users\mihnea.andrei\Work\facial_recognition"
temp_name="temp"
nr_repeats=11
perc_eval=0.2
nr_train=nr_repeats-int(perc_eval*nr_repeats)
horiz_flip=transforms.RandomHorizontalFlip(p=0.5)
color_jitter=transforms.ColorJitter(brightness=0.5,contrast=0.3)
gaussian_blur=transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
adj_sharpness=transforms.RandomAdjustSharpness(sharpness_factor=2)
picture_formats=["jpg","jpeg","png","gif", "pdf", "png", "tiff"]
printing_iter=10**(2)
detector=MTCNN()


def timming(since):
    now=time.time() 
    s=now-since
    m=int(s/60)
    s=s-m*60
    return "%dm %ds"%(m,s)


count_links=1070700
count_people=0
count_faces=0
start=time.time()
#file1=open(r"C:\Users\andre\Work\Facial recognition\wikipedia_face_parser\Print_details.txt","w")
print_data=""
image_name=""

while True:
    prefix="Q"+str(count_links)
    flag=1
    wiki_link="https://www.wikidata.org/wiki/%s"%(prefix)
    json_link="https://www.wikidata.org/wiki/Special:EntityData/%s.json"%(prefix) 
    count_links+=1
    try:
        response=requests.get(json_link)
    except requests.exceptions.Timeout:
        print(100*"-")
        print("Too much time to request wikipedia link %s"%(image_link))
        print(100*"-")
        flag=0
    except requests.exceptions.TooManyRedirects:
        print(100*"-")
        print("Bad wikipedia link %s"%(image_link))
        print(100*"-")
        flag=0
    except requests.exceptions.RequestException as e:
        print(100*"-")
        print("Catastrophic error for wikipaedia link %s"%(image_link))
        print(100*"-")
        flag=0
    if flag==1:
        try: 
            person=json.loads(response.content)
        except ValueError as err:
            flag=0
    if flag==1:
        if prefix in person["entities"].keys(): 
            if "P31" in person["entities"][prefix]["claims"].keys():
                if "mainsnak" in person["entities"][prefix]["claims"]["P31"][0].keys():
                    if "datavalue" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"].keys():
                        if "value" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"].keys():
                            if "id" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"].keys():
                                if person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"]["id"]=="Q5":
                                    #############3
                                    #Obtain the name
                                    ##############3
                                    person_name=""
                                    if "entities" in person.keys():   
                                        if prefix in person["entities"].keys():
                                            if "labels" in person["entities"][prefix].keys(): 
                                                if "en" in person["entities"][prefix]["labels"].keys():
                                                    person_name=person["entities"][prefix]["labels"]["en"]["value"]
                                                    #remove quotes since it gives an error when the image file name when saved
                                                    if '"' in person_name:
                                                        person_name=person_name.replace('"',"")
                                                    if "/" in person_name:
                                                        person_name=person_name.replace('/',"")
                                                    if " "in person_name:
                                                        person_name=person_name.replace(' ',"")
                                                    if "\\" in person_name:
                                                        person_name=person_name.replace(' ',"")
                                                    if "." in person_name:
                                                        person_name=person_name.replace('.',"")
                                                    if "?" in person_name:
                                                        person_name=person_name.replace("?","")
                                                    if "!" in person_name:
                                                        person_name=person_name.replace("!","")
                                                    #remove accents, diacritics
                                                    person_name=unidecode.unidecode(person_name)
                                    ##########
                                    #Obtain the image
                                    #########
                                    if person_name!="":
                                        if "claims" in person["entities"][prefix].keys():
                                            if "P18" in person["entities"][prefix]["claims"].keys():
                                                if "mainsnak" in person["entities"][prefix]["claims"]["P18"][0].keys():
                                                    if "datavalue" in person["entities"][prefix]["claims"]["P18"][0]["mainsnak"].keys():
                                                        if "value" in person["entities"][prefix]["claims"]["P18"][0]["mainsnak"]["datavalue"].keys():
                                                            image_name=person["entities"][prefix]["claims"]["P18"][0]["mainsnak"]["datavalue"]["value"]
                                                            if '"' in image_name:
                                                                image_name=image_name.replace('"','')
                                                            if "\\" in image_name:
                                                                image_name=image_name.replace("\\","")
                                                            image_suffix="_".join(image_name.split(" "))
                                                            image_link="https://www.wikidata.org/wiki/%s#/media/File:%s"%(prefix,image_suffix)
                                                            #image_link = urllib.parse.quote(link,safe=':/')
                                                            try:
                                                                r=requests.get(image_link)
                                                            except requests.exceptions.Timeout:
                                                                print("Too much time to request image link %s"%(image_link))
                                                                flag=0
                                                            except requests.exceptions.TooManyRedirects:
                                                                print("Bad image link %s"%(image_link))
                                                                flag=0
                                                            except requests.exceptions.RequestException as e:
                                                                print("Catastrophic error for image link %s"%(image_link))
                                                                flag=0
                                                            if flag==1:
                                                                #parse HTML code
                                                                soup=BeautifulSoup(r.text,"html.parser")
                                                                #find all images in url (should only be 1 here)
                                                                images=soup.findAll('img')
                                                                if len(images)!=0:
                                                                    image=images[0]
                                                                    try:
                                                                        image_link=image["src"]
                                                                    except:
                                                                        flag=0
                                                                    if flag==1:
                                                                        image_link="https:"+image_link
                                                                        file_path=save_dir+"/"+person_name+".jpg"
                                                                        try:
                                                                            urllib.request.urlretrieve(image_link,file_path)
                                                                        except Exception as e:
                                                                            print("%s for image has error %s, link:%s"%(image_name,e,image_link))
                                                                            flag=0
                                                                        if flag==1:
                                                                            save_to_path=r"C:\Users\mihnea.andrei\Work\facial_recognition\new_faces_images\train"
                                                                            os.mkdir(os.path.join(temp_parent,temp_name))
                                                                            count_people+=1
                                                                            #Need a folder for each person with pictures for that person (in this case 1 like we have for all romanians)
                                                                            #This is because the ImageLOader for resnet reads in the data this way.
                                                                            count_pics=0
                                                                            for i in range(1,nr_repeats+1):
                                                                                try:
                                                                                    image=Image.open(file_path)
                                                                                except (ValueError,UnidentifiedImageError,OSError):
                                                                                    flag=0
                                                                                if flag==1:
                                                                                    save_to_suffix="/".join(file_path.split("/")[-1:]).split(".jpg")[0]+"/"+str(i)+".jpg"
                                                                                    u=random.random()
                                                                                    if 0<=u<0.25:
                                                                                        try:
                                                                                            image=horiz_flip(image)
                                                                                        #except (ValueError, RuntimeError):
                                                                                        except:
                                                                                            pass
                                                                                    if 0.25<=u<0.5:
                                                                                        try:
                                                                                            image=color_jitter(image)
                                                                                        except:
                                                                                            pass
                                                                                    if 0.5<=u<0.75:
                                                                                        try:
                                                                                            image=gaussian_blur(image)
                                                                                        except:
                                                                                            pass
                                                                                    if 0.75<=u<1:
                                                                                        try:
                                                                                            image=adj_sharpness(image)
                                                                                        except:
                                                                                            pass
                                                                                    try:
                                                                                        results=detector.detect(image)
                                                                                    except:
                                                                                        flag=0
                                                                                    if flag==1:
                                                                                        try:
                                                                                            box=results[0][0]
                                                                                        except TypeError:
                                                                                            flag=0
                                                                                        if flag==1:        
                                                                                            for pic_type in picture_formats:
                                                                                                if pic_type in file_path:
                                                                                                    #save_to_path+="/"+save_to_suffix.split("."+pic_type)[0]+".jpg"
                                                                                                    #save_to_temp=temp_dir+"/"+save_to_suffix.split("."+pic_type)[0]+".jpg"
                                                                                                    save_to_temp=temp_dir+"/"+save_to_suffix.split("/")[1]
                                                                                                    #count_pics+=1
                                                                                                    break
                                                                                                else:
                                                                                                    print("Picture format not accepted for %s"%(file_path))
                                                                                                    flag=0
                                                                                            if flag==1:
                                                                                                #extract_face(image,box,save_path=save_to_path)
                                                                                                extract_face(image,box,save_path=save_to_temp)
                                                                                                count_pics+=1
                                                                                                count_faces+=1
                                                                            count_cutoff=int(count_pics*perc_eval)  
                                                                            temp_files=os.listdir(temp_dir)
                                                                            if count_cutoff>=1:
                                                                                counter=0     
                                                                                flag_train=1
                                                                                flag_eval=1
                                                                                for fname in temp_files:
                                                                                    if counter<=count_cutoff:
                                                                                        save_to_path=r"C:\Users\mihnea.andrei\Work\facial_recognition\new_faces_images\eval"
                                                                                        if flag_eval==1:
                                                                                            try:
                                                                                                os.mkdir(os.path.join(save_to_path+"/"+save_to_suffix.split("/")[0]))
                                                                                                flag_eval=0
                                                                                            except (FileExistsError, NotADirectoryError):
                                                                                                flag_eval=0
                                                                                    else:
                                                                                        save_to_path=r"C:\Users\mihnea.andrei\Work\facial_recognition\new_faces_images\train"
                                                                                        if flag_train==1:
                                                                                            try:
                                                                                                os.mkdir(os.path.join(save_to_path+"/"+save_to_suffix.split("/")[0]))
                                                                                                flag_train=0
                                                                                            except FileExistsError:
                                                                                                flag_eval=0
                                                                                    shutil.copy2(os.path.join(temp_dir,fname),os.path.join(save_to_path+"/"+save_to_suffix.split("/")[0]))
                                                                                    counter+=1                                                                                    
                                                                            
                                                                            shutil.rmtree(temp_dir)
                                                                            
                                                                            #for fname in temp_files:
                                                                            #    file_path=os.path.join(temp_dir,fname)
                                                                            #    try:
                                                                            #        if os.path.isfile(file_path) or os.path.islink(file_path):
                                                                            #            os.unlink(file_path)
                                                                            #        elif os.path.isdir(file_path):
                                                                            #            shutil.rmtree(file_path)
                                                                            #    except Exception as e:
                                                                            #        print("Failed to delete %s. Reason: %s"%(file_path,e))
    if count_links%printing_iter==0:
        try:
            print("Downloaded %d people \t Tried %d links \t Created %d faces \t Last person: %s \t %s time"%(count_people,count_links,count_faces,person_name,timming(start)))
        except NameError:
            pass
    

Using cpu device
Downloaded 9 people 	 Tried 1070800 links 	 Created 39 faces 	 Last person: TianHeng 	 0m 53s time
Downloaded 15 people 	 Tried 1070900 links 	 Created 83 faces 	 Last person: YadhiraCarrillo 	 1m 43s time
Downloaded 16 people 	 Tried 1071000 links 	 Created 94 faces 	 Last person: JeanCabannes 	 2m 23s time
Downloaded 27 people 	 Tried 1071100 links 	 Created 153 faces 	 Last person: FrodeJohnsen 	 3m 26s time
Downloaded 28 people 	 Tried 1071200 links 	 Created 153 faces 	 Last person: OmariAldridge 	 4m 6s time
Downloaded 31 people 	 Tried 1071300 links 	 Created 154 faces 	 Last person: TiborSerly 	 4m 48s time
Downloaded 34 people 	 Tried 1071400 links 	 Created 177 faces 	 Last person: XiahouYing 	 5m 32s time
Downloaded 35 people 	 Tried 1071500 links 	 Created 188 faces 	 Last person: ToddHolland 	 6m 15s time
Downloaded 41 people 	 Tried 1071600 links 	 Created 210 faces 	 Last person: ChiamSeeTong 	 7m 4s time
Downloaded 52 people 	 Tried 1071700 links 	 Crea

Downloaded 245 people 	 Tried 1076800 links 	 Created 1627 faces 	 Last person: Rita 	 49m 18s time
Downloaded 248 people 	 Tried 1076900 links 	 Created 1649 faces 	 Last person: JacquesDalbon,SeigneurdeSaintAndre 	 50m 5s time
Downloaded 257 people 	 Tried 1077000 links 	 Created 1707 faces 	 Last person: ChrisBiscoe 	 51m 4s time
Downloaded 270 people 	 Tried 1077100 links 	 Created 1827 faces 	 Last person: ChrisCheney 	 52m 12s time
Downloaded 286 people 	 Tried 1077200 links 	 Created 1947 faces 	 Last person: ChrisFairclough 	 53m 27s time
Downloaded 301 people 	 Tried 1077300 links 	 Created 2021 faces 	 Last person: ChrisHastings 	 54m 34s time
Downloaded 317 people 	 Tried 1077400 links 	 Created 2124 faces 	 Last person: ChrisKelsey 	 55m 47s time
Downloaded 337 people 	 Tried 1077500 links 	 Created 2284 faces 	 Last person: ChrisLewis 	 57m 10s time
Downloaded 354 people 	 Tried 1077600 links 	 Created 2427 faces 	 Last person: ChrisNewman 	 58m 25s time
Downloaded 368 peo

Otto Schwalge 2012-03-30.jpg for image has error <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>, link:https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Otto_Schwalge_2012-03-30.jpg/220px-Otto_Schwalge_2012-03-30.jpg
Downloaded 1235 people 	 Tried 1084100 links 	 Created 9424 faces 	 Last person: SergioManente 	 127m 19s time
Downloaded 1237 people 	 Tried 1084200 links 	 Created 9446 faces 	 Last person: ChristmanGenipperteinga 	 128m 3s time
Downloaded 1244 people 	 Tried 1084300 links 	 Created 9523 faces 	 Last person: ChristofLauer 	 128m 56s time
Downloaded 1253 people 	 Tried 1084400 links 	 Created 9606 faces 	 Last person: ChristofferDebbe 	 129m 51s time
Downloaded 1256 people 	 Tried 1084500 links 	 Created 9639 faces 	 Last person: ChristophAlbrecht 	 130m 34s time
Downloaded 1272 people 	 Tried 10846

Downloaded 1784 people 	 Tried 1090900 links 	 Created 13647 faces 	 Last person:  	 185m 42s time
Downloaded 1788 people 	 Tried 1091000 links 	 Created 13691 faces 	 Last person: AlessandroParisotti 	 186m 32s time
Downloaded 1797 people 	 Tried 1091100 links 	 Created 13746 faces 	 Last person: StavrosKalafatis 	 187m 23s time
Downloaded 1808 people 	 Tried 1091200 links 	 Created 13814 faces 	 Last person: Eleanor,FairMaidofBrittany 	 188m 23s time
Downloaded 1816 people 	 Tried 1091300 links 	 Created 13880 faces 	 Last person: KyleRichardson 	 189m 13s time
Downloaded 1825 people 	 Tried 1091400 links 	 Created 13935 faces 	 Last person: RobertoMalatesta 	 190m 10s time
Downloaded 1833 people 	 Tried 1091500 links 	 Created 13983 faces 	 Last person: LeSeanMcCoy 	 191m 0s time
Downloaded 1840 people 	 Tried 1091600 links 	 Created 14036 faces 	 Last person: RyuzouIshino 	 191m 48s time
Downloaded 1844 people 	 Tried 1091700 links 	 Created 14071 faces 	 Last person: JimAddison 	 

Downloaded 2292 people 	 Tried 1097600 links 	 Created 17134 faces 	 Last person: ClaudiaLux 	 243m 36s time
Downloaded 2308 people 	 Tried 1097700 links 	 Created 17277 faces 	 Last person: IosifHodosiu 	 244m 52s time
Downloaded 2327 people 	 Tried 1097800 links 	 Created 17424 faces 	 Last person: ClaudiaSolzbacher 	 246m 1s time
Downloaded 2338 people 	 Tried 1097900 links 	 Created 17521 faces 	 Last person: ClaudiaWunderlich 	 246m 56s time
Downloaded 2345 people 	 Tried 1098000 links 	 Created 17590 faces 	 Last person: ClaudioBrook 	 247m 47s time
Downloaded 2356 people 	 Tried 1098100 links 	 Created 17700 faces 	 Last person: ClaudioNedden-Boeger 	 248m 43s time
Downloaded 2366 people 	 Tried 1098200 links 	 Created 17786 faces 	 Last person: ClaudiodaLuz 	 249m 39s time
Downloaded 2370 people 	 Tried 1098300 links 	 Created 17820 faces 	 Last person: ClaudiusGosau 	 250m 23s time
Downloaded 2389 people 	 Tried 1098400 links 	 Created 17922 faces 	 Last person: Claus-PeterMar

...

Downloaded 14288 people 	 Tried 1271000 links 	 Created 104084 faces 	 Last person: LouiseGore 	 1726m 24s time
Downloaded 14295 people 	 Tried 1271100 links 	 Created 104117 faces 	 Last person: KellyMatthews 	 1727m 14s time
Downloaded 14302 people 	 Tried 1271200 links 	 Created 104165 faces 	 Last person: JosefOfenbock 	 1728m 4s time
Downloaded 14308 people 	 Tried 1271300 links 	 Created 104217 faces 	 Last person: PhilippWeishaupt 	 1728m 51s time
Downloaded 14314 people 	 Tried 1271400 links 	 Created 104250 faces 	 Last person: LugaidLamderg 	 1729m 40s time
Downloaded 14318 people 	 Tried 1271500 links 	 Created 104261 faces 	 Last person: Bodbchad 	 1730m 26s time
Downloaded 14320 people 	 Tried 1271600 links 	 Created 104280 faces 	 Last person: ToshinariIijima 	 1731m 9s time
Downloaded 14328 people 	 Tried 1271700 links 	 Created 104324 faces 	 Last person: OengusTuirmechTemrach 	 1732m 2s time
Downloaded 14331 people 	 Tried 1271800 links 	 Created 104357 faces 	 Last pe

Downloaded 14742 people 	 Tried 1277500 links 	 Created 107048 faces 	 Last person: HermannSteinacker 	 1783m 13s time
Downloaded 14755 people 	 Tried 1277600 links 	 Created 107161 faces 	 Last person: FelixJosky 	 1784m 12s time
Downloaded 14762 people 	 Tried 1277700 links 	 Created 107204 faces 	 Last person: ElvisBeslagic 	 1785m 1s time
Downloaded 14766 people 	 Tried 1277800 links 	 Created 107237 faces 	 Last person: WilliSchuh 	 1785m 45s time
Downloaded 14772 people 	 Tried 1277900 links 	 Created 107281 faces 	 Last person: EdmundGeisen 	 1786m 32s time
Downloaded 14784 people 	 Tried 1278000 links 	 Created 107326 faces 	 Last person: VladanBatic 	 1787m 26s time
Downloaded 14789 people 	 Tried 1278100 links 	 Created 107348 faces 	 Last person: FelixFalkner 	 1788m 11s time
Downloaded 14794 people 	 Tried 1278200 links 	 Created 107371 faces 	 Last person: KurtLippert 	 1788m 57s time
Downloaded 14801 people 	 Tried 1278300 links 	 Created 107431 faces 	 Last person: Djama

Downloaded 15489 people 	 Tried 1283900 links 	 Created 111921 faces 	 Last person: FilipKasalica 	 1845m 39s time
Downloaded 15508 people 	 Tried 1284000 links 	 Created 112046 faces 	 Last person: EdgarLehmann 	 1846m 50s time
Downloaded 15523 people 	 Tried 1284100 links 	 Created 112161 faces 	 Last person: TuribiusofAstorga 	 1847m 55s time
Downloaded 15538 people 	 Tried 1284200 links 	 Created 112271 faces 	 Last person: EdgarWais 	 1848m 58s time
Downloaded 15561 people 	 Tried 1284300 links 	 Created 112450 faces 	 Last person: EdgardoBerdeguer 	 1850m 19s time
Downloaded 15573 people 	 Tried 1284400 links 	 Created 112549 faces 	 Last person: MattWilson 	 1851m 17s time
----------------------------------------------------------------------------------------------------
Catastrophic error for wikipaedia link https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Basilika_Seckau%2C_Bischofskapelle%2C_Halbfigurenportrait_Bischof_Matthias_Scheit.jpg/220px-Basilika_Seckau%2C_Bi

Downloaded 16422 people 	 Tried 1290600 links 	 Created 118451 faces 	 Last person: JiDelin 	 1914m 28s time
Downloaded 16440 people 	 Tried 1290700 links 	 Created 118528 faces 	 Last person: EduardvonTrzcinski 	 1915m 30s time
Downloaded 16454 people 	 Tried 1290800 links 	 Created 118604 faces 	 Last person: AlainRobidoux 	 1916m 30s time
Downloaded 16468 people 	 Tried 1290900 links 	 Created 118685 faces 	 Last person: GostavonHennigs 	 1917m 30s time
Downloaded 16482 people 	 Tried 1291000 links 	 Created 118776 faces 	 Last person: AndrasSpat 	 1918m 30s time
Downloaded 16497 people 	 Tried 1291100 links 	 Created 118860 faces 	 Last person: EddieGil 	 1919m 31s time
Downloaded 16517 people 	 Tried 1291200 links 	 Created 119043 faces 	 Last person: RichardRatka 	 1920m 47s time
Downloaded 16529 people 	 Tried 1291300 links 	 Created 119110 faces 	 Last person: EdvaldoGoncalvesAmaral 	 1921m 44s time
Downloaded 16549 people 	 Tried 1291400 links 	 Created 119215 faces 	 Last per

KeyboardInterrupt: 